In [65]:
import os 
import json
import pickle
import datetime

import pandas as pd
from pprint import pprint
from bisect import bisect_left as find_prev
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

import xml.etree.ElementTree as xml

from lxml import etree
parser = etree.XMLParser(recover=True)


#tree = ETree.parse("test_xml.xml", parser=parser)
 



strptime = datetime.datetime.strptime

In [66]:
#episodes_numbers = list(map(str, range(175, 185+1)))
episodes_numbers = list(map(str, range(60, 70+1))) #2021 edition for characters Max,Jack,Tanya
#episodes_numbers = list(map(str, range(79, 89+1))) #2021 edition for characters Peggy,Archie
videos_data      = xml.parse('../facerec_segment/eastenders.collection.xml').getroot().findall("./VideoFile")

episodes_filenames = {v.find('id').text : v.find('filename').text
                      for v in videos_data
                      if v.find('id').text in episodes_numbers}
episodes_filenames

{'60': '5245830105934359183.mp4',
 '61': '5248439298566680341.mp4',
 '62': '5251033029316736209.mp4',
 '63': '5253646087419594905.mp4',
 '64': '5256228221757951173.mp4',
 '65': '5258818087037440299.mp4',
 '66': '5264050645694159353.mp4',
 '67': '5266645664934403562.mp4',
 '68': '5269245838135402929.mp4',
 '69': '5271844722846213628.mp4',
 '70': '5274434588125702436.mp4'}

In [67]:
data = []

for line in open('../facerec_segment/eastenders.masterShotReferenceTable.txt', 'r'):
    line = line.strip()
    if not line: continue
    file = line.split()[0].strip()
    if file in episodes_numbers:
        file = episodes_filenames[file]
        _, shot_id, start, end = line.split()
        ref = datetime.datetime(1900, 1, 1)
        start_s = (strptime(start[1:12], '%H:%M:%S:%f') - ref).total_seconds()
        end_s   = (strptime(end[1:12], '%H:%M:%S:%f') - ref).total_seconds()

        data.append({'file':file.replace('mp4', 'xml'), 
                     'shot': shot_id.split('_')[1], 
                     'start_s': start_s, 'end_s': end_s,
                     'start': start[1:12], 'end': end[1:12]})

In [68]:
df = pd.DataFrame(data)

In [5]:
df

,end,end_s,file,shot,start,start_s
0,00:00:20:21,20.21,5245830105934359183.xml,1,00:00:00:00,0.00
1,00:00:23:20,23.20,5245830105934359183.xml,2,00:00:20:22,20.22
2,00:00:25:14,25.14,5245830105934359183.xml,3,00:00:23:21,23.21
3,00:00:26:20,26.20,5245830105934359183.xml,4,00:00:25:15,25.15
4,00:00:30:03,30.03,5245830105934359183.xml,5,00:00:26:21,26.21
5,00:00:32:10,32.10,5245830105934359183.xml,6,00:00:30:04,30.04
6,00:00:35:13,35.13,5245830105934359183.xml,7,00:00:32:11,32.11
7,00:00:38:22,38.22,5245830105934359183.xml,8,00:00:35:14,35.14
8,00:00:51:01,51.01,5245830105934359183.xml,9,00:00:38:23,38.23
9,00:00:54:15,54.15,5245830105934359183.xml,10,00:00:51:02,51.02


In [43]:
df.to_csv('mastershot_timing.csv')

In [69]:
captions = []
transcripts = []

for filename in os.listdir('transcripts'):
    print(filename)
    paragraphs = xml.parse('transcripts/' + filename,parser=parser).getroot().findall('{http://www.w3.org/2006/10/ttaf1}body/*/*')
    for p in paragraphs:
        t = ' '.join(['' if not x.text else x.text.strip() for x in p.findall('*')])
        b = p.attrib['begin']
        e = p.attrib['end']
        if t and (t == t.upper()):
            captions.append({'filename': filename, 'begin':b, 'end':e, 'caption':t})
        else:
            transcripts.append({'filename': filename, 'begin':b, 'end':e, 'transcript':t})

df_captions = pd.DataFrame(captions) 
df_tts = pd.DataFrame(transcripts) 

5300410550331962313.xml
5253646087419594905.xml
5245830105934359183.xml
5271844722846213628.xml
5300433743159556513.xml
5321170704253895214.xml
5269245838135402929.xml
5305617339184877785.xml
5248439298566680341.xml
5264050645694159353.xml
5310822839547632763.xml
5256228221757951173.xml
5266645664934403562.xml
5308226531817199474.xml
5274434588125702436.xml
5318609185758554501.xml
5303026185415228226.xml
5323778608396028180.xml
5313398531435045858.xml
5316007724067368375.xml
5251033029316736209.xml
5258818087037440299.xml


In [35]:
df_tts

,begin,end,filename,transcript
0,00:00:26.517,00:00:28.477,5300410550331962313.xml,I think you'll find that's for me!
1,00:00:32.390,00:00:35.508,5300410550331962313.xml,"You're not gonna get a tip, if that's what yo..."
2,00:00:35.588,00:00:37.188,5300410550331962313.xml,"Just sign here, love, please."
3,00:00:40.818,00:00:45.017,5300410550331962313.xml,"Well, I know that outfit ain't for my benefit..."
4,00:00:45.017,00:00:49.491,5300410550331962313.xml,Why do men think everything revolves around t...
5,00:00:49.491,00:00:51.728,5300410550331962313.xml,First day of the rest of my life.
6,00:00:51.728,00:00:52.928,5300410550331962313.xml,Eh?
7,00:00:55.845,00:00:58.401,5300410550331962313.xml,"Janine, you can't do that!"
8,00:00:58.401,00:01:00.842,5300410550331962313.xml,Watch me.
9,00:01:04.039,00:01:07.632,5300410550331962313.xml,Shopping list. You do the cash and carry. An...


In [36]:
df_captions

,begin,caption,end,filename
0,00:04:04.806,KNOCK AT DOOR,00:04:06.086,5300410550331962313.xml
1,00:04:55.711,I...,00:04:57.949,5300410550331962313.xml
2,00:10:58.844,THEY SHOUT OVER EACH OTHER,00:11:01.565,5300410550331962313.xml
3,00:15:08.262,CRICKET ON TV,00:15:10.179,5300410550331962313.xml
4,00:15:11.459,PHONE RINGS,00:15:13.256,5300410550331962313.xml
5,00:18:53.521,HE WHISTLES,00:18:54.598,5300410550331962313.xml
6,00:25:28.219,SCREAMING AND LAUGHTER,00:25:29.899,5300410550331962313.xml
7,00:29:42.628,MUSIC BLARES,00:29:44.629,5300410550331962313.xml
8,00:30:26.744,BELL RINGS,00:30:28.820,5300410550331962313.xml
9,00:33:32.927,PHONE RINGS,00:33:35.164,5300410550331962313.xml


In [37]:
df_tts

,begin,end,filename,transcript
0,00:00:26.517,00:00:28.477,5300410550331962313.xml,I think you'll find that's for me!
1,00:00:32.390,00:00:35.508,5300410550331962313.xml,"You're not gonna get a tip, if that's what yo..."
2,00:00:35.588,00:00:37.188,5300410550331962313.xml,"Just sign here, love, please."
3,00:00:40.818,00:00:45.017,5300410550331962313.xml,"Well, I know that outfit ain't for my benefit..."
4,00:00:45.017,00:00:49.491,5300410550331962313.xml,Why do men think everything revolves around t...
5,00:00:49.491,00:00:51.728,5300410550331962313.xml,First day of the rest of my life.
6,00:00:51.728,00:00:52.928,5300410550331962313.xml,Eh?
7,00:00:55.845,00:00:58.401,5300410550331962313.xml,"Janine, you can't do that!"
8,00:00:58.401,00:01:00.842,5300410550331962313.xml,Watch me.
9,00:01:04.039,00:01:07.632,5300410550331962313.xml,Shopping list. You do the cash and carry. An...


In [70]:
ref = datetime.datetime(1900, 1, 1)
df_tts['begin_s'] = df_tts['begin'].apply(lambda t: (strptime(t, '%H:%M:%S.%f') - ref).total_seconds())
df_tts['end_s'] = df_tts['end'].apply(lambda t: (strptime(t, '%H:%M:%S.%f') - ref).total_seconds())
df_captions['begin_s'] = df_captions['begin'].apply(lambda t: (strptime(t, '%H:%M:%S.%f') - ref).total_seconds())
df_captions['end_s'] = df_captions['end'].apply(lambda t: (strptime(t, '%H:%M:%S.%f') - ref).total_seconds())

In [24]:
df_tts.head()

,filename,begin,end,transcript,begin_s,end_s
0,5552368364300855101.xml,00:00:30.023,00:00:31.713,"Well, turn it round then.",30.023,31.713
1,5552368364300855101.xml,00:00:33.773,00:00:35.183,"Like it? Yeah, nice one.",33.773,35.183
2,5552368364300855101.xml,00:00:35.183,00:00:37.993,"Cheers, Phil. Well, you better not go showing...",35.183,37.993
3,5552368364300855101.xml,00:00:37.993,00:00:39.773,"hit the canvas before the first bell goes, eh...",37.993,39.773
4,5552368364300855101.xml,00:00:42.543,00:00:44.273,I'll give you a lift to the Deed Poll Office ...,42.543,44.273


In [25]:
df_captions.head()

,filename,begin,end,caption,begin_s,end_s
0,5552368364300855101.xml,00:00:31.713,00:00:33.773,HE CHUCKLES,31.713,33.773
1,5552368364300855101.xml,00:05:53.583,00:05:55.603,MOBILE RINGS,353.583,355.603
2,5552368364300855101.xml,00:08:31.473,00:08:33.063,PHIL CHUCKLES,511.473,513.063
3,5552368364300855101.xml,00:08:36.863,00:08:40.093,"RADIO PLAYS ""CHRISTMAS WRAPPING"" BY THE WAITR...",516.863,520.093
4,5552368364300855101.xml,00:09:00.953,00:09:03.353,SHE CLEARS HER THROAT,540.953,543.353


In [26]:
df.head()

,file,shot,start,end
0,5531550228324592939.xml,1,0.00,0.12
1,5531550228324592939.xml,2,0.13,0.24
2,5531550228324592939.xml,3,1.00,1.18
3,5531550228324592939.xml,4,1.19,2.05
4,5531550228324592939.xml,5,2.06,24.13


In [71]:
df_tts[(df_tts.filename == '5552368364300855101.xml') & \
                         (df_tts.begin_s >= 30) & \
                         (df_tts.begin_s < 31)]

,begin,end,filename,transcript,begin_s,end_s


In [60]:
print(shot)


end                    00:00:20:22
end_s                        20.22
file       5300410550331962313.xml
shot                             1
start                  00:00:00:00
start_s                          0
Name: 0, dtype: object


In [72]:
shot_tts = []
shot_captions = []
for i, shot in tqdm(df.iterrows(), total=len(df)):
    """shot_df_captions = df_captions[(df_captions.filename == shot['file']) & \
                                    (df_captions.begin_s >= shot['start']) & \
                                    (df_captions.begin_s < shot['end'])]
    shot_captions.append({'filename': shot['file'], 'shot_id': shot['shot'], 
                     'begin':shot['start'], 'end': shot['end'], 
                     'caption': ' '.join(shot_df_captions['caption'].values)})"""
    
    
    shot_df_tts = df_tts[(df_tts.filename == shot['file']) & \
                         (df_tts.begin_s >= shot['start_s']) & \
                         (df_tts.begin_s < shot['end_s'])]
    shot_tts.append({'filename': shot['file'], 'shot_id': shot['shot'], 
                     'begin':shot['start'], 'end': shot['end'], 
                     'transcript': ' '.join(shot_df_tts['transcript'].values)})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [74]:
df_shot_tts = pd.DataFrame(shot_tts)
#df_shot_captions = pd.DataFrame(shot_captions)

In [31]:
df_shot_captions[df_shot_captions.caption != '']

,filename,shot_id,begin,end,caption
119,5531550228324592939.xml,120,313.11,315.23,DOOR SLAMS
237,5531550228324592939.xml,238,679.15,688.00,BABY CRIES
246,5531550228324592939.xml,247,746.20,750.16,THEY LAUGH
262,5531550228324592939.xml,263,777.11,779.17,THEY LAUGH
345,5531550228324592939.xml,346,990.16,991.16,CRASHING AND GLASS BREAKING
...,...,...,...,...,...
21152,5555360238519252381.xml,1677,5081.22,5083.10,MACHINE BLEEPS
21154,5555360238519252381.xml,1679,5087.08,5091.13,MACHINE BLEEPS FASTER
21158,5555360238519252381.xml,1683,5095.23,5098.11,SHE SCREAMS ALARM ON MACHINE BLEEPS
21159,5555360238519252381.xml,1684,5098.12,5105.03,SHE STRUGGLES FOR BREATH


In [36]:
df_shot_captions.to_csv('shot-aligned_captions.csv')

In [63]:
df_shot_tts[df_shot_tts.transcript != '']

,begin,end,filename,shot_id,transcript
5,00:00:26:14,00:00:27:16,5300410550331962313.xml,6,I think you'll find that's for me!
8,00:00:31:15,00:00:35:05,5300410550331962313.xml,9,"You're not gonna get a tip, if that's what yo..."
9,00:00:35:06,00:00:36:12,5300410550331962313.xml,10,"Just sign here, love, please."
12,00:00:38:16,00:00:42:24,5300410550331962313.xml,13,"Well, I know that outfit ain't for my benefit..."
13,00:00:43:00,00:00:45:14,5300410550331962313.xml,14,Why do men think everything revolves around t...
16,00:00:49:18,00:00:50:23,5300410550331962313.xml,17,First day of the rest of my life.
17,00:00:50:24,00:00:52:01,5300410550331962313.xml,18,Eh?
19,00:00:53:21,00:00:56:10,5300410550331962313.xml,20,"Janine, you can't do that!"
21,00:00:58:22,00:01:01:07,5300410550331962313.xml,22,Watch me.
24,00:01:03:15,00:01:09:00,5300410550331962313.xml,25,Shopping list. You do the cash and carry. An...


In [75]:
df_shot_tts.to_csv('shot-aligned_transcripts_2021_Max_Jack_Tanya.csv')